In [1]:
import numpy as np
import yfinance as yf
from tqdm import tqdm
import os

# tickers = ["ORCL", "PIRC.MI", "AMZN", "MSFT", "GOOGL", "NVDA", "IBM", "5108.T", "CON.DE", "GT", "5101.T"]
tickers = ["^GSPC", "^IXIC", "FTSEMIB.MI", "^NDX", "^STOXX"]

exchange_map = {
        "GER": "XETRA",
        "NYQ": "NYSE",
        "MIL": "Borsa Italiana",
        "JPX": "JPX",
        "NMS": "NASDAQ",
    }

if not os.path.exists("data"):
    os.makedirs("data")

In [2]:
for tick in tqdm(tickers, total=len(tickers), desc="Downloading data", unit="ticker"):
    ticker = yf.Ticker(tick)
    data = ticker.history(period="10y")
    print(data.columns)
    # data = data[["Open", "High", "Low", "Close", "Volume"]]
    ticker_currency = ticker.info["currency"]
    ticker_company_name = ticker.info["shortName"]
    ticker_exchange = ticker.info["exchange"]
    ticker_tz = ticker.info["exchangeTimezoneName"]
    
    if data.index.tz is None:
        data = data.tz_localize("UTC")
    data = data.tz_convert(ticker_tz)
    data.index = data.index.normalize().tz_localize(None)
    
    data.columns = [f"{col}_{ticker_currency}" for col in data.columns]
    if ticker_currency != "USD":
        pair = f"{ticker.info["currency"]}USD=X"
        fx = yf.Ticker(pair)
        fx_hist = fx.history(start=data.index.min(), end=data.index.max())
        fx_tz = fx.info["exchangeTimezoneName"]
        if fx_hist.index.tz is None:
            fx_hist = fx_hist.tz_localize("UTC")
        fx_hist = fx_hist.tz_convert(fx_tz)
        fx_hist.index = fx_hist.index.normalize().tz_localize(None)
        
        fx_rate = fx_hist[["Close"]].rename(columns={"Close": "FX"})
        fx_rate = fx_rate.reindex(data.index, method="ffill")
        
        data = data.join(fx_rate, how="left")
        data["FX"] = data["FX"].ffill()
        
        for side in ("Open","High","Low","Close","Volume"):
            if side != "Volume":
                data[f"{side}_USD"] = data[f"{side}_{ticker_currency}"] * data["FX"]
            else:
                data[f"{side}_USD"] = np.int64(round(data[f"{side}_{ticker_currency}"] * data["FX"]))
    
    data.to_csv(f"data/{tick}_{ticker_exchange}_{ticker_company_name}.csv")

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')
